# Clustering - K-means, Hierarchical Clustering, DBSCAN 

In this lab we will perform unsupervised classification using clustering algorithms. This will give you an opportunity to explore different clustering methods and different setups for those methods in order to get an intuition as to how the process of performing unsupervised classifiation looks like.

## Dataset

First, we have our dataset. For this lab we will work with a few categories from the Reuters dataset. The code below will download and preprocess the dataset in order to allow us to spend more time on the actual techniques.

In [1]:
import numpy as np
import nltk
nltk.download('reuters', download_dir = './')

[nltk_data] Downloading package reuters to ./...
[nltk_data]   Package reuters is already up-to-date!


True

The previous block code downloads the dataset for us. In order to perform the preprocessing step below, we are required to unzip the reuters folder found at './corpora'.

In [2]:
dataset_folder = './corpora/reuters'


with open(f'{dataset_folder}/cats.txt', 'r') as f:
    annotations = f.readlines()
    
selected_categories = ['sugar', 'livestock', 'jobs', 'ship']
category_to_index = dict(zip(selected_categories, range(len(selected_categories))))

train_texts, train_labels, test_texts, test_labels = [], [], [], []
for ann in annotations:
    ann = ann.rstrip().split()
    
    if not any([category in ann for category in selected_categories]):
        continue
    
    document_text = open(f'{dataset_folder}/{ann[0]}', 'r').read()
    label = category_to_index[
        [category for category in selected_categories if category in ann[1:]][0]
    ]
    
    if 'train' in ann[0]:
        train_texts.append(document_text)
        train_labels.append(label)
    else:
        test_texts.append(document_text)
        test_labels.append(label)
    
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

After this preprocessing we end up with 4 arrays.
- train_texts - which is a list of all the texts in the train dataset
- train_labels - which is a numpy array with the labels of train_texts, corresponding to the chosen categories
- test_texts - which is a list of all the texts in the test dataset
- test_labels - which is a numpy array with the labels of test_texts, corresponding to the chosen categories

## Preprocessing

For this exercise we will use a simple TF-IDF vectorizer from the sklearn library.

Your first task is to compute the train_data and test_data variables, which should be the results of applying the TfidfVectorizer from the sklearn library on our dataset. Use a maximum of 500 features. Fit the vectorizer on the training texts and use it to transform both training and test documents.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 500)
train_data = vectorizer.fit_transform(train_texts)
test_data = vectorizer.transform(test_texts)

print(train_data.shape)
print(test_data.shape)

(434, 500)
(166, 500)


**Note** We use only the training data to fit the vectorizer (build the vocabulary).

## Sanity check

To ensure that everything is okay with our data and preprocessing, and, in order to have a baseline as a reference, fit an SVC on the training data and evaluate it on the test split. Report your results.

In [4]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(train_data, train_labels)
print(np.mean(svc.predict(train_data) == train_labels))
print(np.mean(svc.predict(test_data) == test_labels))

0.988479262672811
0.9457831325301205


## K-Means

The first clustering algorithm we will investigate is the one we've already seen: K-Means.

As a basline, fit the a K-Means model, without any change in parameters on the dataset.

**Note** In otder to have consistency in our results, we will set a random seed for our libraries

In [5]:
random_state = 100
np.random.seed(random_state)

In [6]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 4, random_state = random_state) # we set the seed for the K-means algorithm as well
kmeans.fit(train_data)

predicted_labels = kmeans.predict(train_data)

### Evaluation

Since we work in an unsupervised classification scenario, we will evaluate or model with respect to the class labels that we have. In order to do that, we have to match each cluster to a class (since clusters are not ordered in any particular order). For that, we will compute the confusion matrix (m\[i\]\[j\] = number of samples from class i assigned to cluster j) on the training set and use it do determine the best matching.

In order to perform the matching we will use the linear_sum_assignment method implemented in the scipy.optimize package (https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linear_sum_assignment.html). This method requires a cost function in order to compute the best matching. For that, we will feed the inverse of the confusion matrix, that is 1.0/confusion_matrix.

Once you compute the best matching, translate the cluster labels into class labels and evaluate the accuracy of the model on the training set as well as the test set.

In [7]:
confusion_matrix = np.zeros((4,4))
for train_label, predicted_label in zip(train_labels, predicted_labels):
    confusion_matrix[train_label][predicted_label] += 1
    
from scipy.optimize import linear_sum_assignment
row_ind, col_ind = linear_sum_assignment(1. / confusion_matrix)

translate = dict(zip(col_ind, row_ind))
predicted_labels = np.array([
    translate[label]
    for label in predicted_labels
])

print(np.mean(predicted_labels == train_labels))

0.34101382488479265


/tmp/ipykernel_9662/3849310271.py:6: RuntimeWarning: divide by zero encountered in true_divide
  row_ind, col_ind = linear_sum_assignment(1. / confusion_matrix)


**Note** The linear_sum_assignment procedure returns a list of row indices and a list of column indices. In order to perform the matching, we just need to see which row is associated with each column by zipping the vectors. After replacing the cluster values with the class values, we can check our accuracy on the training set.

**Note** Now, we can evaluate our model on the test set.

In [8]:
test_predictions = np.array([
    translate[label]
    for label in kmeans.predict(test_data)
])
print(np.mean(test_predictions == test_labels))

0.28313253012048195


### Exercises:

Make changes such as the following and take note on how those influence the performance of the model.

1. Consider the fact that the classes are unbalanced. In order to account for that in the cost matrix, divide each row of the confusion matrix by its sum before passing it to the linear_sum_assignment procedure.
2. Insted of passing the inverse of the confusion_matrix as a cost, pass the negative of the confusion matrix
3. Try the k-means++ init for the clustering algorithm
4. Try different values for the n_init parameter
5. Try using PCA or t-sne on your data
6. Try using different amount of features when computing the TF-IDF representations
7. Try using the stopwords provided with the dataset during the TF-IDF vectorization (nltk_data/corpora/reuters/stopwords)
9. Try using a different vectorization procedure
10. Try normalizing your data

**Note** I will showcase a few of these changes and assess their impact on the performance of our model. First I will only build a vocabulary of 100 words, forcing irrelevant tokens out. Secondly, I will use -confusion_matrix instead of 1./confusion_matrix as a cost for the linear_sum_assignment algorithm, in order to avoid dividing by 0.

In [9]:
vectorizer = TfidfVectorizer(max_features = 100)
train_data = vectorizer.fit_transform(train_texts)
test_data = vectorizer.transform(test_texts)

kmeans = KMeans(n_clusters = 4, random_state = random_state)
kmeans.fit(train_data)

predicted_labels = kmeans.predict(train_data)

confusion_matrix = np.zeros((4,4))
for train_label, predicted_label in zip(train_labels, predicted_labels):
    confusion_matrix[train_label][predicted_label] + 1
    
row_ind, col_ind = linear_sum_assignment(- confusion_matrix)

translate = dict(zip(col_ind, row_ind))
predicted_labels = np.array([
    translate[label]
    for label in predicted_labels
])

print(np.mean(predicted_labels == train_labels))

test_predictions = np.array([
    translate[label]
    for label in kmeans.predict(test_data)
])
print(np.mean(test_predictions == test_labels))

0.619815668202765
0.6265060240963856


**Note** These changes alone got us an improvement of 34.3% on the test set. Let's also try to change the way we compute cluster-class correlations. As we've noticed, the clusters are not balanced, meaning a an overlap of 10 documents between a class and a cluster might mean a lot if the cluster has 20 document in total, and not much if the cluster contains 200 documents. Thus, let us divide each column of the confusion matrix by the sum of the column, that is, for each cluster, divide the number of documents overlapping with each individual class, by the number of documents in the cluster.

In [10]:
vectorizer = TfidfVectorizer(max_features = 100)
train_data = vectorizer.fit_transform(train_texts)
test_data = vectorizer.transform(test_texts)

kmeans = KMeans(n_clusters = 4, random_state = 100)
kmeans.fit(train_data)

predicted_labels = kmeans.predict(train_data)

confusion_matrix = np.zeros((4,4))
for train_label, predicted_label in zip(train_labels, predicted_labels):
    confusion_matrix[train_label][predicted_label] += 1
    
for col in range(4):
    confusion_matrix[:, col] /= np.sum(confusion_matrix[:, col])
    
row_ind, col_ind = linear_sum_assignment(- confusion_matrix)

translate = dict(zip(col_ind, row_ind))
predicted_labels = np.array([
    translate[label]
    for label in predicted_labels
])

print(np.mean(predicted_labels == train_labels))

test_predictions = np.array([
    translate[label]
    for label in kmeans.predict(test_data)
])
print(np.mean(test_predictions == test_labels))

0.716589861751152
0.7048192771084337


**Note** This change got us an additional 7.8% increase in accuracy.

### Consider the realistic scenario

Consider an actual project where you are given a set of documents collected from the web, that do not have any human annotations. You are tasked to group those documents semantically.

Choose one of the models developed in the previous phase.

1. Look through the cluster texts in order to get a sense of what each cluster represents.
2. For each cluster extract the most important/characteristic words and see what they point to
    - you can do this by evaluating documents that are closer to the center of the cluster
    - you can do this by running statistics on the entire cluster
    - you can use different measures for the importance of a word (number of distinct documents within which the word appears, tfidf for the documents closer to the center, etc)
    
Use this exercise in order to get a sense of the data and to get a sense of how the clustering algorithm grouped them and what those groups represent.

**Note** This is a more subjective exercise. Students are encouraged to actually look through this information by hand. For the purposes of this lab, we will only illustrate extracting the most important words out of the entirety of the cluster.

In [11]:
## sorting the vocabulary words by indexes
# print( vectorizer.vocabulary_ )
vocab = sorted(list(vectorizer.vocabulary_.items()), key = lambda x: x[1])
# print(vocab)
vocab = np.array([word for word, index in vocab])


for cluster_n in range(4):
    cluster_texts = [
        train_texts[i]
        for i in range(len(train_texts))
        if predicted_labels[i] == cluster_n
    ]
#     print(cluster_texts[0])
    
    # computing the mean tfidf of the cluster documents
    cluster_tfidf = np.array(np.mean(
        train_data[predicted_labels == cluster_n], axis = 0
    ))[0] # morphing a np.matrix to a numpy array
    best_word_indexes = np.argsort(cluster_tfidf)[80:] # getting the top 20 words based on the mean tfidf value
    best_words = vocab[best_word_indexes]
    print(best_words)
    
    

['export' 'commission' 'year' 'was' 'is' 'from' 'mln' 'at' 'and' 'for'
 'said' 'in' 'traders' 'ec' '000' 'of' 'to' 'tonnes' 'the' 'sugar']
['from' 'dlrs' 'said' 'japan' 'about' '1986' 'april' 'of' 'at' 'in' 'week'
 'to' 'for' 'beef' 'the' 'year' 'and' 'tonnes' '1987' '000']
['march' 'tonnes' 'was' '000' 'at' 'and' 'said' 'week' 'year' '1986'
 'from' 'of' 'to' 'january' 'february' 'unemployment' 'mln' 'pct' 'in'
 'the']
['its' 'be' 'strike' 'was' 'port' 'is' 'ships' 'will' 'it' 'that' 'at'
 'by' 'on' 'for' 'said' 'in' 'and' 'of' 'to' 'the']


**Note** Note the word "sugar" as the most importat word of the first cluster. We also notice "beef", for the second, "unemployment" for the third and "ships" for the forth.

## Hierarchical Clustering

We will continue the investigation of our problem using a different clustering procedure, namely agglomerative clustering. For that we will use the implemenation provided by the scipy library (https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html).

### Exercises

Repeat evaluating the different setups presented in the previous set of exercises in order to investigate the current clustering method. In addition:

1. Evaluate different linking methods
2. Evaluate different metrics (such as cosine)

And see how they impact the performance and learning process.

In [12]:
train_data = train_data.toarray() # from scipy sparse matrix to array
test_data = test_data.toarray()

In [13]:
from sklearn.cluster import AgglomerativeClustering

hierarchical = AgglomerativeClustering(n_clusters = 4, linkage = 'complete')
predicted_labels = hierarchical.fit_predict(train_data)

confusion_matrix = np.zeros((4,4))
for train_label, predicted_label in zip(train_labels, predicted_labels):
    confusion_matrix[train_label][predicted_label] += 1
    
for col in range(4):
    confusion_matrix[:, col] /= np.sum(confusion_matrix[:, col])
    
row_ind, col_ind = linear_sum_assignment(- confusion_matrix)

translate = dict(zip(col_ind, row_ind))
predicted_labels = np.array([
    translate[label]
    for label in predicted_labels
])

print(np.mean(predicted_labels == train_labels))

0.576036866359447


**Note** We will evaluate the training performance by replicating the linkage method on the new samples. In our scenario, for a given new sample, we will take each cluster and compute the maximum distance between the new sample and the cluster samples. The cluster closest in terms of the maximum distance will be the one to which the new sample is assigned.

In [14]:
hierarchical = AgglomerativeClustering(n_clusters = 4, linkage = 'complete')
predicted_labels = hierarchical.fit_predict(train_data)

cluster_samples = dict()
for cluster_n in range(4):
    cluster_samples[cluster_n] = train_data[predicted_labels == cluster_n] # collecting cluster samples
    
test_predictions = []
for test_sample in test_data:
    max_distances = []
    for cluster_n in range(4):
        distances = np.sum(np.square(cluster_samples[cluster_n] - test_sample), axis = 1) # computing distances towards each cluster sample, similar to the KNN lab
        cluster_max_distance = np.max(distances) # getting the maximum distance for the  "complate" linkage
        max_distances.append(cluster_max_distance)
    test_predictions.append(np.argmin(max_distances)) # computing assignment by getting the closest cluster
test_predictions = np.array(test_predictions)


# computing performance
confusion_matrix = np.zeros((4,4))
for train_label, predicted_label in zip(train_labels, predicted_labels):
    confusion_matrix[train_label][predicted_label] += 1
    
for col in range(4):
    confusion_matrix[:, col] /= np.sum(confusion_matrix[:, col])
    
row_ind, col_ind = linear_sum_assignment(- confusion_matrix)

translate = dict(zip(col_ind, row_ind))
predicted_labels = np.array([
    translate[label]
    for label in predicted_labels
])

print(np.mean(predicted_labels == train_labels))

test_predictions = np.array([
    translate[label]
    for label in test_predictions
])
print(np.mean(test_predictions == test_labels))

0.576036866359447
0.40963855421686746


## DBSCAN

Finally, we will make use of the DBSCAN algorithm in order to cluster the data (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).

### Exercises

Investigate the DBSCAN model. In addition to the previous setups:

1. Evaluate the distribution of distances between samples in the training data and use them as a reference point when deciding the parameters
2. Perform a grid search on the parameters
3. Use a different association rule for evaluation, which allows multiple clusters to pe assigned to a single class, for instance, based on the confusion matrix, assign each cluster to the class with which it has the most samples in common

In [15]:
from sklearn.cluster import DBSCAN

distance = np.zeros((len(train_labels), len(train_labels)))
for i, train_sample_1 in enumerate(train_data):
    for j, train_sample_2 in enumerate(train_data):
        distance[i][j] = np.sqrt(np.sum(np.square(train_sample_1 - train_sample_2)))

In [16]:
print(np.min(distance[distance != 0]), np.mean(distance), np.median(distance), np.max(distance))

0.08713524907467607 1.1305081980945428 1.143984815769202 1.4142135623730954


In [17]:
for d in np.linspace(0.1, 1, 22):
    dbscan = DBSCAN(eps = d).fit(train_data)
    n_clusters = np.max(dbscan.labels_) + 1
    print(n_clusters, d)

2 0.1
2 0.14285714285714285
2 0.18571428571428572
2 0.22857142857142856
3 0.27142857142857146
3 0.3142857142857143
3 0.3571428571428571
4 0.4
4 0.44285714285714284
5 0.48571428571428577
5 0.5285714285714286
5 0.5714285714285714
8 0.6142857142857142
9 0.6571428571428571
8 0.7
6 0.7428571428571429
6 0.7857142857142857
5 0.8285714285714285
3 0.8714285714285714
1 0.9142857142857143
1 0.9571428571428572
1 1.0


**Note** After summarily evaluating the distances and the amount of clusters we can get based on various values for the *eps* parameter, we can perform an evaluation. Let's say we can't obtain the exact amount of clusters we desire. For instance, let's take *eps* = 0.55 and work with our 5 clusters. Naturally, we can assign each new sample from the test set to the closest cluster in term of the minimum distance between the test sample and the cluster samples. Afterwards we can compute our evaluation metric by assigning each cluster to the class with which it has the most documents in common. We can do that by computing the argmax on each column of the confusion matrix.

In [18]:
dbscan = DBSCAN(eps = 0.55).fit(train_data)
predicted_labels = dbscan.labels_
n_clusters = np.max(dbscan.labels_) + 1

cluster_samples = dict()
for cluster_n in range(n_clusters):
    cluster_samples[cluster_n] = train_data[predicted_labels == cluster_n]
    
test_predictions = []
for test_sample in test_data:
    min_distances = []
    for cluster_n in range(n_clusters):
        distances = np.sum(np.square(cluster_samples[cluster_n] - test_sample), axis = 1)
        cluster_min_distance = np.min(distances)
        min_distances.append(cluster_min_distance)
    test_predictions.append(np.argmin(min_distances))
test_predictions = np.array(test_predictions)


confusion_matrix = np.zeros((4, n_clusters))
for train_label, predicted_label in zip(train_labels, predicted_labels):
    confusion_matrix[train_label][predicted_label] += 1
    
translate = dict()
for i in range(n_clusters):
    translate[i] = np.argmax(confusion_matrix[:, i])

test_predictions = np.array([
    translate[label]
    for label in test_predictions
])
print(np.mean(test_predictions == test_labels))

0.5542168674698795


**Note** In the end, with our setup, we got a 55.4% performance on the test set, which surpasses the one obtained using the hierarchical clustering method.